In [1]:
import numpy as np

def is_positive_definite(A):
    return np.all(np.linalg.eigvals(A) > 0)

def cholesky_LU(A):
    """
    Compute Cholesky decomposition of a positive definite matrix A = L @ L.T
    Returns lower triangular matrix L
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A)
    
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square")
        
    if not np.allclose(A, A.T) or not is_positive_definite(A):
        raise ValueError("Matrix must be symmetric positive definite")

    n = A.shape[0]
    L = np.zeros_like(A, dtype=float)

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                L[i,i] = np.sqrt(A[i,i] - np.sum(L[i,:i]**2))
            else:
                L[i,j] = (A[i,j] - np.sum(L[i,:j]*L[j,:j])) / L[j,j]
    
    return L

# Example usage:
if __name__ == "__main__":
    A = np.array([[4, 2, -2], 
                  [2, 10, 2], 
                  [-2, 2, 5]])
    L = cholesky_LU(A)
    L_true = np.linalg.cholesky(A, upper=False)
    print("L:\n", L)
    print("Verification L@L.T:\n", L @ L.T)
    print("L:\n", L_true)

L:
 [[ 2.          0.          0.        ]
 [ 1.          3.          0.        ]
 [-1.          1.          1.73205081]]
Verification L@L.T:
 [[ 4.  2. -2.]
 [ 2. 10.  2.]
 [-2.  2.  5.]]
L:
 [[ 2.          0.          0.        ]
 [ 1.          3.          0.        ]
 [-1.          1.          1.73205081]]


In [2]:

def BC_decomp(A):
    """
    Performs Doolittle LU decomposition of matrix A where:
    - B is lower triangular
    - C is upper triangular with ones on diagonal
    - A = B @ C
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)
    
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square")

    B = np.zeros((n, n))
    C = np.eye(n, n)
    
    for i in range(n):
        # Calculate B elements
        for j in range(i + 1):
            sum_b = sum(B[i,k] * C[k,j] for k in range(j))
            B[i,j] = A[i,j] - sum_b
            
        # Calculate C elements
        for j in range(i + 1, n):
            sum_c = sum(B[i,k] * C[k,j] for k in range(i))
            C[i,j] = (A[i,j] - sum_c) / B[i,i]
            # C[i,j] = (A[i,j] - sum_c)
            
    return B, C

dim = 4
A = np.random.default_rng().random(size=(dim, dim), dtype=np.float64)
for i in range(dim):
  for j in range(dim):
    if i < j:
      A[i,j] = A[j, i]

B, C = BC_decomp(A)
print("B (lower):\n", B)
print("\nC (upper):\n", C)
print("\nVerification B@C:\n", B @ C)
print("\nOriginal A:\n", A)

B (lower):
 [[ 0.85660072  0.          0.          0.        ]
 [ 0.76979425 -0.42227185  0.          0.        ]
 [ 0.87820923 -0.5793023   0.67923782  0.        ]
 [ 0.48636802  0.14310548  0.17484793  0.70359684]]

C (upper):
 [[ 1.          0.89866168  1.02522588  0.56778847]
 [ 0.          1.          1.37187052 -0.3388942 ]
 [ 0.          0.          1.          0.25741784]
 [ 0.          0.          0.          1.        ]]

Verification B@C:
 [[0.85660072 0.76979425 0.87820923 0.48636802]
 [0.76979425 0.26951275 0.20991069 0.58018578]
 [0.87820923 0.20991069 0.78487291 0.8698072 ]
 [0.48636802 0.58018578 0.8698072  0.97626235]]

Original A:
 [[0.85660072 0.76979425 0.87820923 0.48636802]
 [0.76979425 0.26951275 0.20991069 0.58018578]
 [0.87820923 0.20991069 0.78487291 0.8698072 ]
 [0.48636802 0.58018578 0.8698072  0.97626235]]


In [4]:
def solve_bc_system(B: np.ndarray, C: np.ndarray, f: np.ndarray) -> np.ndarray:
    """
    Solve system Ax=f where A=BC using:
    1. By=f -> find y
    2. Cx=y -> find x 
    """
    if not isinstance(B, np.ndarray) or not isinstance(C, np.ndarray):
        raise ValueError("Inputs must be numpy arrays")
        
    n = B.shape[0]
    if B.shape[1] != n or C.shape[0] != n or C.shape[1] != n:
        raise ValueError("Invalid matrix dimensions")

    # Step 1: Solve By = f
    y = np.zeros(n)
    for i in range(n):
        y[i] = f[i]
        for k in range(i):
            y[i] -= B[i,k] * y[k]
        y[i] /= B[i,i]

    # Step 2: Solve Cx = y 
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for k in range(i+1, n):
            x[i] -= C[i,k] * x[k]
        # Note: C diagonal elements are 1
        
    return x

f_test = np.ones(dim)

print(solve_bc_system(*BC_decomp(A), f_test))

print(np.linalg.solve(A, f_test))

[ 0.66334359  0.59222929 -0.42797593  0.72319128]
[ 0.66334359  0.59222929 -0.42797593  0.72319128]
